In [1]:
#!pip install geopy
#!pip install folium
import pandas as pd
import statsmodels as sm
import matplotlib.pyplot as plt
import json
import folium
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap

In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
df_50k = pd.read_csv('noislamophobia-dataset-50k.csv')
df_75k = pd.read_csv('noislamophobia-dataset-75k.csv')
df = pd.concat([df_50k,df_75k])
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,8,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,_id,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,...,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,full_text,truncated,user,withheld_in_countries
0,ObjectId(59dbede4e6e465a2d67a1062),NaN,NaN,Sun Oct 22 10:34:06 +0000 2017,"{""hashtags"":[{""text"":""BanIslam"",""indices"":[93,...",NaN,0,False,NaN,9.220484e+17,...,NaN,4,False,"{""created_at"":""Fri Oct 20 09:33:45 +0000 2017""...","<a href=""http://twitter.com/download/iphone"" r...",RT @Private34349909: @AmyMek @Dab7One 1/ @real...,NaN,False,"{""id"":8.009259051368776e+17,""id_str"":""80092590...",NaN
1,ObjectId(59dbede4e6e465a2d67a106c),NaN,NaN,Sun Oct 22 10:13:40 +0000 2017,"{""hashtags"":[{""text"":""muslim"",""indices"":[16,23...",NaN,0,False,NaN,9.220433e+17,...,NaN,10,False,"{""created_at"":""Sat Oct 21 18:37:07 +0000 2017""...","<a href=""http://twitter.com/download/android"" ...",RT @ensine: All #muslim rulers were savages bc...,NaN,False,"{""id"":375089876,""id_str"":""375089876"",""name"":""द...",NaN
2,ObjectId(59dbf266e6e465a2d67a4810),NaN,NaN,Sun Oct 22 07:50:47 +0000 2017,"{""hashtags"":[{""text"":""Raqqa"",""indices"":[37,43]...",NaN,0,False,NaN,9.220073e+17,...,NaN,172,False,"{""created_at"":""Fri Oct 20 23:15:45 +0000 2017""...","<a href=""http://twitter.com/download/android"" ...",RT @SLandinSoCal: Liberated Women of #Raqqa‼️R...,NaN,False,"{""id"":7.092975641898926e+17,""id_str"":""70929756...",NaN
3,ObjectId(59dbf266e6e465a2d67a4828),NaN,NaN,Sun Oct 22 10:38:23 +0000 2017,"{""hashtags"":[],""symbols"":[],""user_mentions"":[{...",NaN,0,False,NaN,9.220495e+17,...,NaN,1,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@Stormtroepen @cdavandaag @sybrandbuma @gertja...,NaN,True,"{""id"":1340408646,""id_str"":""1340408646"",""name"":...",NaN
4,ObjectId(59e8e630e6e465a2d6238c9c),NaN,NaN,Sun Oct 22 08:59:53 +0000 2017,"{""hashtags"":[{""text"":""RT"",""indices"":[51,54]},{...",NaN,0,False,NaN,9.220247e+17,...,NaN,25,False,"{""created_at"":""Sun Oct 22 01:40:55 +0000 2017""...","<a href=""http://twitter.com/download/android"" ...",RT @PoliticalIslam: Sharia at odds with Articl...,NaN,False,"{""id"":150595824,""id_str"":""150595824"",""name"":""🇮...","[""DE""]"


In [3]:
place_df = df[df["place"].notna()].astype('str')
place_df = place_df["place"]
print(place_df)

94       {"id":"1d68da80ca90416d","url":"https://api.tw...
206      {"id":"53cef5332ac9d7d0","url":"https://api.tw...
638      {"id":"f632697d33274211","url":"https://api.tw...
642      {"id":"5635c19c2b5078d1","url":"https://api.tw...
997      {"id":"a81f9ed24c15d6af","url":"https://api.tw...
                               ...                        
73381    {"id":"2a8a74486cd0d519","url":"https://api.tw...
73915    {"id":"243cc16f6417a167","url":"https://api.tw...
73972    {"id":"8e9665cec9370f0f","url":"https://api.tw...
74221    {"id":"67d92742f1ebf307","url":"https://api.tw...
74229    {"id":"ad0818e2fb208dde","url":"https://api.tw...
Name: place, Length: 713, dtype: object


In [4]:
places = place_df.apply(json.loads).apply(pd.Series)
places['full_name'] = places['full_name'].str.replace(', USA', '')
places['location'] = places['full_name'] + ", " + places['country']
places.head()

,id,url,place_type,name,full_name,country_code,country,contained_within,bounding_box,attributes,location
94,1d68da80ca90416d,https://api.twitter.com/1.1/geo/id/1d68da80ca9...,city,Kawartha Lakes,"Kawartha Lakes, Ontario",CA,Canada,[],"{'type': 'Polygon', 'coordinates': [[[-79.2086...",{},"Kawartha Lakes, Ontario, Canada"
206,53cef5332ac9d7d0,https://api.twitter.com/1.1/geo/id/53cef5332ac...,city,Waunfawr,"Waunfawr, Wales",GB,United Kingdom,[],"{'type': 'Polygon', 'coordinates': [[[-4.21700...",{},"Waunfawr, Wales, United Kingdom"
638,f632697d33274211,https://api.twitter.com/1.1/geo/id/f632697d332...,city,Mount Airy,"Mount Airy, MD",US,United States,[],"{'type': 'Polygon', 'coordinates': [[[-77.1957...",{},"Mount Airy, MD, United States"
642,5635c19c2b5078d1,https://api.twitter.com/1.1/geo/id/5635c19c2b5...,admin,Virginia,Virginia,US,United States,[],"{'type': 'Polygon', 'coordinates': [[[-83.6752...",{},"Virginia, United States"
997,a81f9ed24c15d6af,https://api.twitter.com/1.1/geo/id/a81f9ed24c1...,city,Grand Rapids,"Grand Rapids, MI",US,United States,[],"{'type': 'Polygon', 'coordinates': [[[-85.7514...",{},"Grand Rapids, MI, United States"


In [5]:
locator = Nominatim(user_agent='myGeocoder')
m = folium.Map(location=[20,0], zoom_start=2)
data = []
for index, row in places.iterrows():
    location = locator.geocode(row['location'])
    if (location == None):
        location = locator.geocode(row['country'])
        
    data.append([location.latitude, location.longitude, 1])
    
    folium.Circle(
        radius=400,
        location=[location.latitude, location.longitude],
        popup=row["name"],
        color="crimson",
        fill=False,
    ).add_to(m)
places_copy = pd.DataFrame(data, columns=['latitude', 'longitude', 'count'])
m.save('50kdotmap.html')
m

In [6]:
n = folium.Map(location=[20,0], zoom_start=2)
HeatMap(data=places_copy[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=10, max_zoom=10).add_to(n)
n.save('50kheatmap.html')
n

In [12]:
total_entries = len(df.index)
print(len(df.index))
print(len(places.index))
location_entries = len(places.index)
print(str(100*(location_entries/total_entries)) + "% of total entries have location information available.")
#Studies demonstrate that approximately 0.85% of tweets are geotagged: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4636345/"
#Study looked at a sample of 113 million tweets. As such, we can be 99% confident that that this is a statistically significant difference
#What is making this percentage so low? Most likely, there is an abundance of bots posting negative about Islam

125384
713
0.568653097683915% of total entries have location information available.
Studies demonstrate that approximately 0.85% of tweets are geotagged: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4636345/
